# LGE-SNU DS 고급과정 HMM 실습

HMM 관련 패키지 중 하나인 hmmlearn 패키지를 사용해보자.
hmmlearn 패키지에서는 아래와 같은 3가지 형태의 hmm을 제공한다.


|          |  |
|----------|-----------------|
| <br>GaussianHMM | <br> HMM with Gaussian emissions.| 
| <br>GMMHMM   |<br> HMM with Gaussian mixture emissions.           | 
| <br>MultinomialHMM |<br> HMM with multinomial (discrete) emissions.            |


강의자료에는 observation이 discrete 인 경우만 소개했지만, continuous 한 경우도 GMMHMM 또는 GaussianHMM 을 이용해서 쉽게 분석할 수 있다. 본 실습에서는 MultinomialHMM와 GaussianHMM을 이용한 분석을 소개한다.


## 1. 관측값이 이산확률변수인 경우
https://en.wikipedia.org/wiki/Template:HMM_example 에 소개된 아래 예시를 이용하여 모형을 분석해보자.

Bob 은 그날의 날씨 (Rainy, Sunny) 에 따라 산책(Walk), 쇼핑(shop), 방청소(clean) 중 하나의 행동을 취한다.
Alice 는 Bob 이 그날그날 어떠한 행동을 취했는지만을 알고 있으며, 이를 통해 날씨에 대해 추측을 하고자 한다.
이를 Hidden Markov chain 의 관점에서 보면 날씨 (Rainy, Sunny)는 숨겨진 정보로써 은닉변수에 해당하며, Bob이 취하는 행동은 각 날씨에서 특정 출력확률을 따라 관측되는 값으로 이해할 수 있다.

본 분석에서는 아래와 같은 형태의 Markov chain을 가정한다. 은닉변수의 초기상태에 대한 확률분포 (0.6,0.4)는 날씨의 경향에 대한 Alice의 믿음으로 이해할 수 있으며 전이확률과 출력확률은 아래와 같은 값을 갖는다고 하자. 예를 들어, 전날에 비(Rainy)가 왔다면 다음날 날씨가 맑을(Sunny) 확률은 0.3이며 맑은 날씨에 Bob이 쇼핑을 할 확률은 0.3 이라고 생각할 수 있다.

![title](400px-HMMGraph.svg.png)

이제 hmmlearn 내의 MultinomialHMM 를 이용하여 실제 모형을 만들고 그로부터 관측값을 얻어보자.

In [39]:
from __future__ import division
import numpy as np
from hmmlearn import hmm
import random

states = ('Rainy', 'Sunny')
n_states = len(states)
observations = ('walk', 'shop', 'clean')
 
# 실제 모형
model_true = hmm.MultinomialHMM(n_components=n_states, init_params = '', params='')# n_components : 은닉변수가 가질 수 있는 값의 개수

# 은닉변수의 초기상태에 대한 확률분포
model_true.startprob_ = np.array([0.6, 0.4])
 
# 전이확률 행렬
model_true.transmat_ = np.array([
  [0.7, 0.3],
  [0.4, 0.6]
])

# 출력확률 행렬
model_true.emissionprob_ = np.array([
  [0.1, 0.4, 0.5],
  [0.6, 0.3, 0.1]
])

In [44]:
random.seed(100)
bob_says, hidden_state = model_true.sample(10) # 크기 10 인 관측값을 랜덤하게 출력
print(bob_says.T)
print("Bob says:", ", ".join(map(lambda x: observations[int(x)], bob_says)))
print("Hidden state:", ", ".join(map(lambda x: states[int(x)], hidden_state)))

[[1 2 1 1 2 0 0 1 2 2]]
Bob says: shop, clean, shop, shop, clean, walk, walk, shop, clean, clean
Hidden state: Rainy, Rainy, Rainy, Rainy, Rainy, Sunny, Sunny, Rainy, Rainy, Rainy


### 학습(Learning) 과 디코딩(Decoding)
hmmlearn 패키지에서는 $\textbf{fit}$ 함수를 이용하여 HMM의 모수 (전이확률($A$), 출력확률($B$), 은닉변수의 초기상태에 대한 확률($\pi$)) 를 추정할 수 있으며,
$\textbf{decode}$ 함수를 이용하여 최적의 은닉변수열을 찾을 수 있다.


In [41]:
model_pred = hmm.MultinomialHMM(n_components=n_states, n_iter = 100, tol = 1.0e-2)
model_pred = model_pred.fit(bob_says) # 추정된 모형

추정된 모수들은 각각 아래와 같다.

In [42]:
model_pred.transmat_, model_pred.emissionprob_,model_pred.startprob_

(array([[0.79763159, 0.20236841],
        [0.2482735 , 0.7517265 ]]),
 array([[3.28937174e-01, 5.75624295e-06, 6.71057070e-01],
        [9.88632124e-03, 9.89778533e-01, 3.35145945e-04]]),
 array([1.000000e+00, 2.315775e-25]))

In [43]:
logprob, hidden_state_pred = model_pred.decode(bob_says, algorithm="viterbi") # viterbi 알고리즘 이용
print("Bob says:", ", ".join(map(lambda x: observations[int(x)], bob_says)))
print("Prediction of hidden state:", ", ".join(map(lambda x: states[int(x)], hidden_state_pred)))
print("Accuracy:",np.mean(hidden_state==hidden_state_pred)) # 실제 날씨와 일치하는 비율

Bob says: clean, clean, shop, shop, shop, shop, walk, clean, walk, clean
Prediction of hidden state: Rainy, Rainy, Sunny, Sunny, Sunny, Sunny, Rainy, Rainy, Rainy, Rainy
Accuracy: 0.7


## 2. 관측값이 연속확률변수인 경우

- 관측값이 이산확률변수인 경우 출력확률 $P(O_t | X_t=j)$ 이 이산확률분포를 나타내었지만, 관측값이 연속확률변수인 경우에는 이러한 접근이 불가능하다.
- 이 경우 출력확률을 연속확률분포로 이해하며, 출력확률행렬 대신 분포의 모수를 추정해야 한다.
- 본 실습에서는 각 출력확률이 Gaussian 인 경우에 대해 다루고자 한다.

![title](hmm.png)

In [159]:
# 실제 모형
model_true = hmm.GaussianHMM(n_components=3) # n_components : 은닉변수가 가질 수 있는 값의 개수

# 은닉변수의 초기상태에 대한 확률분포
model_true.startprob_ = np.array([0.5, 0.2, 0.3])

# 전이확률 행렬
model_true.transmat_ = np.array([[0.5, 0.3, 0.2],
                     [0.2, 0.6, 0.2],
                     [0.0, 0.5, 0.5]])

# 각 출력확률분포의 평균
model_true.means_ = np.array([[-1.0],[1.0],[3.0]])

# 각 출력확률분포의 분산
model_true.covars_  = np.array([[1],[1],[1]])



In [160]:
observation, hidden_state = model_true.sample(15) # 크기 15 인 관측값을 랜덤하게 출력

print("Observation:",observation.T)
print("Hidden state:",hidden_state)

Observation: [[-1.36175474 -2.75987118 -1.87504116  1.05532855  2.63004027  2.30849104
  -1.53944295 -0.24155532 -1.88565559  0.99030076  1.50669879  1.43002059
   2.22594218  0.02765109  2.75094937]]
Hidden state: [0 0 0 1 2 1 1 0 0 1 1 1 2 1 2]


In [161]:
model_pred = hmm.GaussianHMM(n_components=3, n_iter = 100, tol = 1.0e-2)
model_pred = model_pred.fit(observation) # 추정된 모형

추정된 모수들은 각각 아래와 같다.

In [162]:
model_pred.transmat_, model_pred.means_, model_pred.covars_, model_pred.startprob_

(array([[6.66656668e-01, 3.33343331e-01, 8.60485658e-10],
        [3.41893847e-05, 2.86413525e-06, 9.99962946e-01],
        [2.46796973e-01, 4.80553895e-01, 2.72649132e-01]]),
 array([[-1.61036352],
        [ 0.86853591],
        [ 2.27557764]]),
 array([[[0.57007781]],
 
        [[0.27007548]],
 
        [[0.19669409]]]),
 array([1.00000000e+00, 1.56842472e-37, 4.31765581e-98]))

In [163]:
logprob, hidden_state_pred = model_pred.decode(observation, algorithm="viterbi") # viterbi 알고리즘 이용
print("Observation:",observation.T)
print("Prediction of hidden state:", hidden_state_pred)
print("Accuracy:",np.mean(hidden_state==hidden_state_pred)) # 실제 은닉변수와 일치하는 비율

Observation: [[-1.36175474 -2.75987118 -1.87504116  1.05532855  2.63004027  2.30849104
  -1.53944295 -0.24155532 -1.88565559  0.99030076  1.50669879  1.43002059
   2.22594218  0.02765109  2.75094937]]
Prediction of hidden state: [0 0 0 1 2 2 0 0 0 1 2 1 2 1 2]
Accuracy: 0.8
